### 검증세트

In [2]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine-date')

In [3]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [6]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input,train_target,test_size=0.2,random_state=42)

In [7]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [9]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state =42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [10]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00760937, 0.00789595, 0.00601506, 0.00206423, 0.        ]), 'score_time': array([0.00139952, 0.00102568, 0.00096059, 0.        , 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


- cross_validate() 함수는 평가할 모델 객체를 첫번째 매개 변수로 전달. fit_time, score_time, test_score 키를 가진 딕셔너리 반환. 각 키마다 5개의 숫자 담겨있음.

In [11]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


- 교차검증을 수해앟면 모델에서 얻을 수 있는 최상의 검증 점수를 가늠해 볼 수 있음.
- cross_validat()는 훈련 세트를 준비했기 때문에 따로 섞을 필요 없음. 교차 검증을 할 때 훈련 세트를 섞으려면 분할기 지정.(spliter)

In [12]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [13]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores =  cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


- 그리드 서치

In [14]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}

In [15]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)

In [16]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [17]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [18]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [19]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [20]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


1. 탐색할 매개변수 지정
2. 훈련세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다.이 조합은 그리드 서치 객체에 저장됨. 
3. 그리드 서치는 최상의 매개변수에서 전체 훈련세트를 사용해 최종 모델을 훈련함. 이 모델도 그리드 서치 객체에 저장됨. 

- min_impurity_decreas 는 노드를 분할하기 위한 불순도 감소 최소량 지정
- max_depth로 트리의 깊이 제한. 
- min_samples_split으로 노드를 나누기 위한 최소 샘플 수 고름

In [21]:
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),'max_depth':range(5,20,1),'min_samples_split':range(2,100,10)}

In [22]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [23]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
# p.251re